<a href="https://colab.research.google.com/github/mathewpolonsky/Marketplace-Item-Matching/blob/main/Working_with_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Импорт библиотек

In [ ]:
import json

import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


---

## Работа с датасетом

In [ ]:
!gdown 1m836LjMM6c2zRFB3HrhuaqUvuiccRW6S

Downloading...
From: https://drive.google.com/uc?id=1m836LjMM6c2zRFB3HrhuaqUvuiccRW6S
To: /content/datasets.zip
100% 1.24G/1.24G [00:10<00:00, 117MB/s]


In [ ]:
!unzip datasets.zip

Archive:  datasets.zip
   creating: datasets/
  inflating: datasets/test_data.parquet  
  inflating: datasets/test_pairs_wo_target.parquet  
  inflating: datasets/train_data.parquet  
  inflating: datasets/train_pairs.parquet  


---

### Загрузка

In [ ]:
dataset = pd.read_parquet("datasets/train_pairs.parquet")
etl = pd.read_parquet("datasets/train_data.parquet")

In [ ]:
dataset.head(2)

,target,variantid1,variantid2
0,0.0,51197862,51198054
1,1.0,53062686,536165289


In [ ]:
etl.head(2)

,variantid,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64,characteristic_attributes_mapping
0,51195767,"Удлинитель Партнер-Электро ПВС 2х0,75 ГОСТ,6A,...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[оранжевый],None,"[[0.04603629, 0.18839523, -0.09973055, -0.6636...","[-0.47045058, 0.67237014, 0.48984158, -0.54485...","{""Номинальный ток, А"":[""10""],""Цвет товара"":[""о..."
1,53565809,Магнитный кабель USB 2.0 A (m) - USB Type-C (m...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Кабели ...",[красный],"[[0.26863545, -0.3130674, 0.29023397, 0.073978...","[[1.1471839, -0.665361, 0.7745614, 0.26716197,...","[-0.6575592, 0.6522429, 0.5426037, -0.54347897...","{""Конструктивные особенности"":[""Магнитная конс..."


### Предобработка

In [ ]:
etl["categories"][:5].to_numpy()

array(['{"1": "EPG", "2": "Электроника", "3": "Сетевые фильтры, разветвители и удлинители", "4": "Сетевой фильтр, удлинитель, разветвитель"}',
       '{"1": "EPG", "2": "Электроника", "3": "Кабели и переходники", "4": "Кабель"}',
       '{"1": "EPG", "2": "Электроника", "3": "Оптические приборы", "4": "Микроскоп"}',
       '{"1": "EPG", "2": "Электроника", "3": "Смартфоны, планшеты, мобильные телефоны", "4": "Мобильный телефон"}',
       '{"1": "EPG", "2": "Электроника", "3": "Штативы и головки", "4": "Штатив"}'],
      dtype=object)

#### Полный текст о товаре

In [ ]:
def attrs_format(attributes):
    return '\n'.join([f'{i[0]}: {", ".join(i[1])}' for i in attributes.items()])

def make_full_text(row):
	full_text = [f"name: {row['name']}"]

	if row['color_parsed'] is not None:
		full_text.append(f"color: {', '.join(row['color_parsed'])}")

	full_text.append(f"cat3: {json.loads(row['categories'])['3']}")
	full_text.append(f"cat4: {json.loads(row['categories'])['4']}")

	if row['characteristic_attributes_mapping'] is not None:
		full_text.append(f"{attrs_format(json.loads(row['characteristic_attributes_mapping']))}")

	return "\n".join(full_text)
	

columns = ["name", "color_parsed", "categories", "characteristic_attributes_mapping"]

etl["full_text"] = etl[columns].apply(lambda x: make_full_text(x), axis=1)

print(etl.full_text[7])

name: Плетеный нейлоновый ремешок для часов Apple Watch series 1-7 и Эпл Вотч SE 38-40-41 mm / Эластичный тканевый монобраслет Solo Loop (155 mm), Красный
color: красный
cat3: Смарт-часы
cat4: Ремешок для смарт-часов
Рекомендовано для: Apple
Цвет товара: красный
Размер циферблата: 38mm, 40mm, 41mm
Длина ремешка, мм: 155
Образец цвета: https://cdn1.ozone.ru/s3/multimedia-f/6043129923.jpg
Бренд: Pastila
Вес товара, г: 12
Гарантийный срок: 14 дней, при сохранении товарного вида
Материал браслета/ремешка: Текстиль
Размеры, мм: 155
Название цвета: красный
Страна-изготовитель: Россия
Модель браслета/умных часов: Apple Watch SE 40mm, Apple Watch Series 1 38mm, Apple Watch Series 2 38mm, Apple Watch Series 3 38mm, Apple Watch Series 4 40mm, Apple Watch Series 5 40mm, Apple Watch Series 6 40mm, Apple Watch Series 7 41mm
Комплектация: Ремешок - 1 шт., упаковка - 1 шт.


#### Merge и сохранение

Train

In [ ]:
columns2remove = ['categories', 'characteristic_attributes_mapping',
                  'pic_embeddings_resnet_v1', 'name', 'name_bert_64', 'color_parsed']

etl = etl.drop(columns2remove, axis=1)

etl.head(3)

,variantid,main_pic_embeddings_resnet_v1,full_text
0,51195767,"[[0.04603629, 0.18839523, -0.09973055, -0.6636...","name: Удлинитель Партнер-Электро ПВС 2х0,75 ГО..."
1,53565809,"[[1.1471839, -0.665361, 0.7745614, 0.26716197,...",name: Магнитный кабель USB 2.0 A (m) - USB Typ...
2,56763357,"[[-0.90570974, 1.0296293, 1.0769907, 0.27746, ...","name: Набор микропрепаратов Konus 25: ""Клетки ..."


In [ ]:
train = (
    dataset
    .merge(
        etl
        .add_suffix('1'),
        on="variantid1"
    )
    .merge(
        etl
        .add_suffix('2'),
        on="variantid2"
    )
)

In [ ]:
new_index = list(train.columns)

# Изменяем имена колонок например с full_text1 на full_text_1
for i in range(3, len(new_index)):
    name = new_index[i]
    
    if name.endswith("1"):
        new_index[i] = name[:-1] + "_1"

    else:
        new_index[i] = name[:-1] + "_2"

train.columns = new_index

train.head(2)

,target,variantid1,variantid2,main_pic_embeddings_resnet_v1_1,full_text_1,main_pic_embeddings_resnet_v1_2,full_text_2
0,0.0,51197862,51198054,"[[-0.4304909, -0.49474272, -0.46439183, -0.060...",name: Удлинитель TDM Electric Люкс УЛ05В 5 м (...,"[[-0.42941108, -0.5129398, -0.4753536, -0.0677...",name: Удлинитель TDM Electric Люкс УЛ05В 1.5 м...
1,0.0,51197862,51199884,"[[-0.4304909, -0.49474272, -0.46439183, -0.060...",name: Удлинитель TDM Electric Люкс УЛ05В 5 м (...,"[[-0.43180764, -0.49580905, -0.5062628, -0.130...",name: Удлинитель TDM Electric Люкс УЛ05В 3 м (...


In [ ]:
train.to_parquet("drive/MyDrive/datasets/merged_train.parquet", engine='pyarrow', index=False)

In [ ]:
train[['target', 'variantid1', 'variantid2', 'full_text_1', 'full_text_2']].to_parquet(
    "datasets/merged_train4nlp.parquet", engine='pyarrow', index=False)

---
Test

In [ ]:
test_pairs = pd.read_parquet("datasets/test_pairs_wo_target.parquet")
test_etl = pd.read_parquet("datasets/test_data.parquet")

In [ ]:
test_pairs

,variantid1,variantid2,cat3_grouped
0,52076340,290590137,Батарейки и аккумуляторы
1,64525522,204128919,"Смартфоны, планшеты, мобильные телефоны"
2,77243372,479860557,Кабели и переходники
3,86065820,540678372,Устройство ручного ввода
4,91566575,258840506,"Смартфоны, планшеты, мобильные телефоны"
...,...,...,...
18079,666998614,667074522,Оперативная память
18080,670036240,670048449,Зарядные устройства и док-станции
18081,670284509,684323809,Смарт-часы
18082,692172005,704805270,rest


In [ ]:
columns = ["name", "color_parsed", "categories", "characteristic_attributes_mapping"]

test_etl["full_text"] = test_etl[columns].apply(lambda x: make_full_text(x), axis=1)

columns2remove = ['categories', 'characteristic_attributes_mapping',
                  'pic_embeddings_resnet_v1', 'name', 'name_bert_64', 'color_parsed']

test_pairs = test_pairs.drop('cat3_grouped', axis=1)

test_etl = test_etl.drop(columns2remove, axis=1)

test_etl.head(3)

,variantid,main_pic_embeddings_resnet_v1,full_text
0,51201254,"[[0.38310742, -0.7876679, 0.5018278, 0.2090071...",name: Колодка TDM Electric четырехместная без ...
1,77151532,"[[0.50964713, 0.7958329, -1.4113188, 0.1999381...",name: Клавиатура черная с черной рамкой для 25...
2,89664856,"[[0.9958085, -0.113175124, -0.7623152, -0.9164...","name: 15.6"" Игровой ноутбук Acer Predator Heli..."


In [ ]:
test = (
    test_pairs
    .merge(
        test_etl
        .add_suffix('1'),
        on="variantid1"
    )
    .merge(
        test_etl
        .add_suffix('2'),
        on="variantid2"
    )
)

In [ ]:
new_index = list(test.columns)

for i in range(2, len(new_index)):
    name = new_index[i]
    
    if name.endswith("1"):
        new_index[i] = name[:-1] + "_1"

    else:
        new_index[i] = name[:-1] + "_2"

test.columns = new_index

test.head(2)

,variantid1,variantid2,main_pic_embeddings_resnet_v1_1,full_text_1,main_pic_embeddings_resnet_v1_2,full_text_2
0,52076340,290590137,"[[0.04763528, -0.20136409, 0.29605597, 0.26453...",name: Батарейка AAA щелочная Perfeo LR03/10BL ...,"[[0.06223978, -0.16145544, 0.26409012, 0.24271...",name: Батарейка AAA щелочная Perfeo LR03/2BL m...
1,64525522,204128919,"[[-0.27325493, -0.6696304, 0.027148303, 0.0785...","name: Смартфон Ulefone Armor X5 3/32 ГБ, черны...","[[-0.15358369, -0.8256463, -0.054863703, 0.453...","name: Смартфон Ulefone Armor X3 2/32 ГБ, черны..."


In [ ]:
test.to_parquet("drive/MyDrive/datasets/merged_test.parquet", engine='pyarrow', index=False)

In [ ]:
test[['variantid1', 'variantid2', 'full_text_1', 'full_text_2']].to_parquet(
    "drive/MyDrive/datasets/merged_test4nlp.parquet", engine='pyarrow', index=False
)